Noms des coéquipers: **<span style="color:red">ICI</span>**

# Parcours de groupes

$G$ étant un groupe fini, il arrive que l'on doive algorithmiquement 
parcourir tous les éléments de $G$ (l'exemple typique étant: énumérer toutes les permutations d'un ensemble à $n$ éléments), ce qui peut en général se faire de plusieurs façons. Une d'entre elles est de se donner un ensemble $S \subseteq G$ 
d'éléments distingués de $G$ et de considérer le graphe dirigé $\Gamma$, dit *de Cayley*, associé à la paire $(G,S)$, pour lequel

$$ V(\Gamma) = G \quad \text{et} \quad A(\Gamma) = \{ \, (x, \, x \cdot s) 
 \mid x \in G, \, s \in S \, \}: $$

les sommets sont les éléments de $G$, et il y a un arc de $x$ à $y$ 
lorsqu'il existe un élément distingué $s \in S$ pour lequel $y = s \cdot x$.

Nous allons utiliser dans ce TP notre implémentation des permutations de la dernière séance (une est fournie, vous pouvez utiliser la vôtre si elle est fonctionnelle !).

In [1]:
load("Permutation.sage")

-= Classe Permutation (PROF) chargée =-


In [2]:
Permutation([1,4,3,2]) * Cycle([1,2]) == Permutation([4,1,3,2])  # vérification

True

## Échauffement

### 1 &ndash; Un petit graphe

Voici le graphe de Cayley associé à la paire $G = \mathcal{S}_4$, $S = \{ \color{blue}{(1 
\, 2)}, \, \color{red}{(1 \, 2 \, 3 \, 4)} \}$:

In [3]:
a = Cycle([1,2])
b = Cycle([1,2,3,4])

pts = { "1234": ( 0, 1, 2), "2341": ( 1, 0, 2), "3412": ( 0,-1, 2), "4123": (-1, 0, 2),
        "2134": ( 0, 2, 1), "1342": (-1, 2, 0), "3142": (-2, 1, 0), "1423": (-2, 0, 1),
        "4231": (-2,-1, 0), "2431": (-1,-2, 0), "4312": ( 0,-2, 1), "3124": ( 1,-2, 0),
        "1243": ( 0,-2,-1), "2143": ( 0,-1,-2), "2314": (-2, 0,-1), "3214": (-1, 0,-2),
        "1432": ( 1, 0,-2), "4321": ( 0, 1,-2), "4132": ( 2, 0,-1), "3241": ( 2, 0, 1),
        "2413": ( 2, 1, 0), "4213": ( 1, 2, 0), "3421": ( 0, 2,-1), "1324": ( 2,-1, 0) }

def str_repr(s):
    "représentation sur une ligne de la permutation s"
    return "".join(str(x) for x in s.data)

img = Graphics()
for p in pts:
    s = Permutation([ int(x) for x in p ])
    img += text3d(p, 1.1*vector(pts[p]), color='black')
    img += arrow( pts[p], pts[str_repr(s*a)], color="blue" )
    img += arrow( pts[p], pts[str_repr(s*b)], color="red" )
show(img, frame=false)

Graphics3d Object

<b>Questions:</b>

a) D'après le graphe, quel est le sommet $\sigma$ le plus éloigné de la permutation identité $\mathrm{id}$ et à quelle distance $d$ se trouve-t-il ? Donner en observant le graphe une expression de longueur $d$ permettant de l'exprimer comme produit des générateurs et vérifier calculatoirement que c'est bien le cas.

b) Vérifier explicitement (par force brute) qu'aucune expression de la forme $a_1 \cdots a_k$ avec les $a_i \in S$ et $k < d $ n'est égale à $\sigma$. NB: cela revient à effectuer un parcours en largeur du graphe à partir du sommet $\mathrm{id}$.

### 2 &ndash; Un moyen graphe

Pour un graphe plus gros, typiquement on n'y voit rien et on doit se rabattre sur le calcul. Considérons par exemple le graphe de Cayley $\Gamma$ associé à la paire $G = \mathcal{S}_6$, $S = \{ (1 \, 2) (3 \, 4), \, (1 \, 2 \, 4 \, 5) (3 \, 6) \}$.

<b>À faire:</b> implémenter un parcours en largeur de ce graphe en partant de la permutation identité. Notez qu'il vaut mieux être paresseux et n'évaluer un sommet que lorsqu'on le rencontre pour la première fois: le graphe en tant que tel n'a pas besoin d'exister en tant que structure de données avant d'être parcouru.

Vous pourrez ensuite répondre à ces questions:

a) Le graphe est-il connexe ?

b) Quel est son diamètre ? (distance maximale entre deux sommets)

c) À quelle distance de l'identité se trouve le sommet $(1 \, 2 \, 3) (4 \, 5 \, 6)$ ? Donner une expression minimale de cet élément en terme des générateurs.

## 3 &ndash; Bonus: Application aux jeux de permutations

Plusieurs jeux combinatoires peuvent être vus comme des instances du problème « trouver un chemin dans un graphe de Cayley ».

Exemples:

- jeu de taquin
- cube Rubik
- <a href="https://www.jaapsch.net/puzzles/topspin.htm">TopSpin</a>
- ...

Dans tous ces cas, on a:

- un ensemble $X$ de configurations du jeu (souvent représentées par des fonctions d'un ensemble $P$ de positions vers un ensemble $C$ de pièces);
- un groupe $G \subseteq \mathcal{S}_P$ de mouvements possibles, qui agit par conséquent sur $X$;
- un ensemble $S \subseteq G$ de mouvements élémentaires permis (ex. tourner une face d'un cube Rubik);
- un graphe de Cayley $\Gamma$ associé au triplet $(X, G, S)$: les sommets sont les éléments de $X$, et il y a une arête de $x$ vers $y$ lorsque $y = s \cdot x$ où $s \in S$
- une configuration spéciale $x_0 \in X$ dite _résolue_

et le but du jeu, étant donné une configuration $x \in X$, est de trouver un chemin dans $\Gamma$ de $x$ vers $x_0$ (si un tel chemin existe !).

Je vous invite à modéliser votre jeu de permutation préféré ainsi: préciser les ensembles $P$, $C$, $X$ et $S$ et commencer l'exploration du graphe au voisinage de la configuration résolue $x_0$.

(NB: cette exploration systématique devient vite impossible parce que le groupe $G$ a tendance à être <a href="https://www.therubikzone.com/number-of-combinations/">rapidement très gros</a> &ndash; par contre pour le cube Rubik $2 \times 2 \times 2$ si on s'y prend bien on peut tout cartographier, il n'y a « que » quelques millions de sommets dans la composante connexe de la configuration résolue...)